In [1]:
import requests
import json
import time
import pandas as pd
import mysql.connector
from IPython.display import clear_output
from datetime import datetime
from config import api_football_key, conn_host, conn_database, conn_user, conn_password

In [2]:
league_id = 71 # BR League
start_season = 2022
end_season = 2023

now = time.time()

In [3]:
def connect_to_db():
    return mysql.connector.connect(host=conn_host, 
                                     database=conn_database,
                                     user=conn_user,
                                     password=conn_password)

def execute_query(query, read_only = True):
    resp = None
    try:
        db = connect_to_db()
        if read_only:
            resp = pd.read_sql_query(query, db)
        else:
            mycursor = db.cursor()
            mycursor.execute(query)

            db.commit()
    except Exception as e:
        print(e)
    db.close()
    return resp

In [4]:
# Table creation queries
execute_query("CREATE TABLE IF NOT EXISTS leagues (id INT NOT NULL, name VARCHAR(50) NOT NULL, PRIMARY KEY (id))", False)
execute_query("CREATE TABLE IF NOT EXISTS teams (id INT NOT NULL, name VARCHAR(50) NOT NULL, PRIMARY KEY (id))", False)
execute_query("CREATE TABLE IF NOT EXISTS matches (id INT NOT NULL, date DATETIME NOT NULL, league_id INT NOT NULL, season INT NOT NULL, home_id INT NOT NULL, away_id INT NOT NULL, home_score INT NULL, away_score INT NULL, PRIMARY KEY (id), FOREIGN KEY(home_id) REFERENCES teams (id), FOREIGN KEY(away_id) REFERENCES teams (id), FOREIGN KEY(league_id) REFERENCES leagues (id))", False)

In [5]:
def get_league_season_fixtures(season):
    headers = {
        'X-RapidAPI-Key': api_football_key,
        'X-RapidAPI-Host': 'api-football-v1.p.rapidapi.com'
    }
    response = requests.get(f"https://api-football-v1.p.rapidapi.com/v3/fixtures?league={league_id}&season={season}", headers=headers)
    response_parsed = json.loads(response.text)
    return [fixture for fixture in response_parsed['response'] if fixture['fixture']['timestamp'] < now]

In [6]:
def add_match_info_to_db(fixture):
    fixture_id, league_id, league_name, fixture_date, season, home_id, home_name, away_id, away_name, home_score, away_score = fixture['fixture']['id'], fixture['league']['id'], f"{fixture['league']['name']} ({fixture['league']['country']})", fixture['fixture']['timestamp'], fixture['league']['season'], fixture['teams']['home']['id'], fixture['teams']['home']['name'], fixture['teams']['away']['id'], fixture['teams']['away']['name'], fixture['goals']['home'], fixture['goals']['away']
    fixture_date_converted = datetime.fromtimestamp(fixture_date).strftime('%Y-%m-%d %H:%M:%S')
    execute_query(f"INSERT IGNORE INTO leagues (id, name) VALUES ({league_id}, '{league_name}')", False)
    execute_query(f"INSERT IGNORE INTO teams (id, name) VALUES ({home_id}, '{home_name}')", False)
    execute_query(f"INSERT IGNORE INTO teams (id, name) VALUES ({away_id}, '{away_name}')", False)
    execute_query(f"INSERT IGNORE INTO matches (id, date, league_id, season, home_id, away_id, home_score, away_score) VALUES ({fixture_id}, '{fixture_date_converted}', {league_id}, {season}, {home_id}, {away_id}, '{home_score}', '{away_score}')", False)
#     print(f"{fixture['teams']['home']['name']} {fixture['goals']['home']} x {fixture['goals']['away']} {fixture['teams']['away']['name']}")

In [7]:
for season in range(start_season, end_season):
    fixtures = get_league_season_fixtures(season)
    for index, fixture in enumerate(fixtures):
        clear_output(wait=True)
        print(f"Loading fixtures for the {season} season: {index}/{len(fixtures)}")
        add_match_info_to_db(fixture)

Loading fixtures for the 2022 season: 169/170


In [11]:
fixtures_df = execute_query(f"SELECT m.id, m.date, m.season, l.name AS league, ht.name as home_team, at.name as away_team, m.home_score, m.away_score FROM matches AS m INNER JOIN teams AS ht ON (m.home_id = ht.id) INNER JOIN teams AS at ON (m.away_id = at.id) INNER JOIN leagues AS l ON (m.league_id = l.id) WHERE m.season = 2022 ORDER BY m.date DESC")

C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [13]:
fixtures_df.head()

,id,date,season,league,home_team,away_team,home_score,away_score
0,838154,2022-07-18 20:00:00,2022,Serie A (Brazil),Palmeiras,Cuiaba,1,0
1,838151,2022-07-17 19:00:00,2022,Serie A (Brazil),America MG,Bragantino,0,3
2,838155,2022-07-17 18:00:00,2022,Serie A (Brazil),Botafogo RJ,Atletico-MG,0,1
3,838158,2022-07-17 18:00:00,2022,Serie A (Brazil),Atletico GO,Fortaleza,0,1
4,838153,2022-07-17 16:00:00,2022,Serie A (Brazil),Sao Paulo,Fluminense,2,2
